<a href="https://colab.research.google.com/github/Pedroct06/Coloriza-o-de-fotos/blob/main/UpgradeCor_PedroTorres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Usei o dataset de colorização que eu encontrei no kaggle: https://www.kaggle.com/datasets/aayush9753/image-colorization-dataset/data

In [ ]:
import tensorflow as tf
import kagglehub
import numpy as np
from tensorflow.keras import layers, models
import cv2
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

Defino o tamanho das imagens para ser 128x128, e o caminho de onde está o meu dataSet

In [ ]:
Img_size = 128
dataset_path = kagglehub.dataset_download("aayush9753/image-colorization-dataset")
caminho_real = os.path.join(dataset_path, 'data')

Inicializo duas listas vazias, no qual a primeira vai receber as imagens em preto e branco, que representa a entrada. Enquanto a outra recebe colorida, para guardar a saida. Além disso percorri as imagens presentes dentro da pasta de entrada, criando o caminho correto para serem processadas, transformando para 128x128. Após isso, eu converto de RGB para LAB, e separo a imagem em tres variáveis de canais: L, A, B.
Divido a minha variável L por 255, para deixar os valores entre 0 e 1, e faço o reshape pois o canal L precisa ter o espaço para o canal de cor(AB).
Após isso, eu junto os canais A e B, gerando uma imagem com 2 canais de cores, e faço a normalização subtraindo 128 e dividindo por 128, para ficar entre -1 e 1.
Após tudo isso, adiciono nas duas listas criadas anteriormente, e retorno transformando para array de numpy, para poder utilizar no treinamento

In [ ]:
def carregar_imagem(input, esperado):
    Icinza = []
    Icor = []
    arquivos = os.listdir(input)
    for nome in arquivos:
        caminho_saida = os.path.join(esperado, nome)

        img_bgr = cv2.imread(caminho_saida)
        img_bgr = cv2.resize(img_bgr, (Img_size, Img_size))

        img_lab = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LAB)

        l, a, b = cv2.split(img_lab)

        img_input_l = l / 255.0
        img_input_l = img_input_l.reshape(Img_size, Img_size, 1)

        ab = np.stack((a, b), axis=2)

        img_out_ab = (ab - 128.0) / 128.0

        Icinza.append(img_input_l)
        Icor.append(img_out_ab)
    return np.array(Icinza), np.array(Icor)

Pego os arquivos pelos caminhos

In [ ]:
treino_cinza = os.path.join(caminho_real, 'train_black')
treino_cor = os.path.join(caminho_real, 'train_color')
teste_cinza = os.path.join(caminho_real, 'test_black')
teste_cor = os.path.join(caminho_real, 'test_color')

Pego os dois arrays retornados na função de carregar a imagem. Separando em treino e teste

In [ ]:
X_train, Y_train = carregar_imagem(treino_cinza, treino_cor)
X_test, Y_test = carregar_imagem(teste_cinza, teste_cor)

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D, Concatenate
from tensorflow.keras.models import Model

Defino a minha rede neural, peguei de modelo uma rede do tipo U-net que funciona como um funil.
Primeiro passo eu recebi a minha imagem no modelo 128x128x1
Depois eu vou diminuindo o tamanho da imagem pela metade, enquanto dobro a quantidade de detalhes e uso a função de ativação relu(C1,C2,C3)
B é o local onde a imagem está menor, porém com muitos detalhes. Após isso, comeco a aumentar a imagem novamente (U1), aumentando de 16x16 para 32x32, e concatenando com informação do mesmo tamanho(C1,C2,C3), para retomar a espacialização perdida quando foi diminuindo.

Na saída só temos dois filtros, para prever A e B, e uso a função de ativação tanh, pois normalizei os dados entre -1 e 1. E preciso de números negativos para representar as cores, logo não poderia usar a sigmoid nem relu
Depois crio o modelo com o input e output criados

In [ ]:
def rede_neural():
    inputs = Input(shape = (Img_size, Img_size, 1))

    c1 = Conv2D(64, (3, 3), activation='relu', padding='same', strides =2)(inputs)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same', strides = 2)(c1)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same', strides =2 )(c2)

    b = Conv2D(512, (3, 3), activation='relu', padding='same')(c3)

    u1 = Conv2DTranspose(256, (3, 3), activation='relu', padding='same', strides = 2)(b)
    merge = Concatenate()([u1, c2])
    c4 = Conv2D(256, (3, 3), activation='relu', padding='same')(merge)

    u2= Conv2DTranspose(128, (3, 3), activation='relu', padding='same', strides = 2)(c4)
    merge = Concatenate()([u2, c1])
    c5 = Conv2D(128, (3, 3), activation='relu', padding='same')(merge)

    u3 = Conv2DTranspose(64, (3, 3), activation='relu', padding='same', strides = 2)(c5)
    merge = Concatenate()([u3, inputs])
    c6 = Conv2D(64, (3, 3), activation='relu', padding='same')(merge)

    outputs = Conv2D(2, (3, 3), activation='tanh', padding='same')(c6)
    model = Model(inputs=[inputs], outputs=[outputs])
    return model

Compilei o meu modelo, utilizando otimizador adam, pois se adapta a muitas situações. E utilizei a função de custo MAE, pois demonstrou um desempenho melhor do que a MSE(Calcula o erro absoluto médio, porém sem elevar ao quadrado), realizei o teste das duas

In [ ]:
model = rede_neural()
model.compile(optimizer='adam', loss='mae')

Realizei o treino com os dados de treino, utilizei 100 épocas(quantidade de vezes que meu conjunto de treino vai ser visto), os conjuntos de pesos só serão ajustados após passar por 32 fotos, na qual ela calcula o erro e passa pras outras 32. Utilizei também a validation_data para ver se a minha rede neural não está decorando para os casos de treino (overfitting)

In [ ]:
model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_data=(X_test, Y_test))

Aqui eu fiz a função para carregar a minha imagem e plotar o gráfico com a minha colorida, e com a original. Ela faz o inverso da normalização dos dados, para poder formar uma imagem

In [ ]:
def ver_imagem(i):
  L_input = X_test[i]
  ab_predict = model.predict(L_input.reshape(1, 128, 128, 1))[0]
  L_ajustado = L_input * 100.0
  ab_ajustado = (ab_predict * 128.0) * 3
  LAB = np.concatenate([L_ajustado, ab_ajustado], axis=2)
  RGB = cv2.cvtColor(LAB.astype('float32'), cv2.COLOR_LAB2RGB)
  RGB = np.clip(RGB, 0, 1)

  ab_real = Y_test[i]
  ab_real_ajustado = ab_real * 128.0
  LAB_real = np.concatenate([L_input * 100.0, ab_real_ajustado], axis=2)
  RGB_real = cv2.cvtColor(LAB_real.astype('float32'), cv2.COLOR_LAB2RGB)
  RGB_real = np.clip(RGB_real, 0, 1)

  plt.figure(figsize = (12,6))

  plt.subplot(1, 2, 1)
  plt.title("imagem colorida")
  plt.imshow(RGB)
  plt.axis('off')

  plt.subplot(1, 2, 2)
  plt.title("imagem colorida real")
  plt.imshow(RGB_real)
  plt.axis('off')

  plt.show()

In [ ]:
ver_imagem(1)